# Análise exploratória e testes amostral - linha 844

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória
- (3) Tabela pós-reprocessamento

Preparar o ambiente:

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [2]:
servico = "844"

#### 1 - Importar e tratar os dados da amostra

In [3]:
amostra = pd.read_excel('../data/raw/Relatorio de Viagens das linhas 605_LECD50. 663 e 844.xlsx')

# padronizar as colunas da amostra 
amostra = amostra.rename(
    columns={"Data": "data",
             "Hora_Inicio": "hora_inicio",
             "Linha": "servico",
             "Veiculo": "id_veiculo",
             "Hora_Fim": "hora_fim",
             "Sentido": "sentido"
             }
    )

# setar tipos de dados e criar colunas com o datetime
amostra['servico'] = amostra['servico'].astype(str)
amostra['data'] = amostra['data'].astype(str)
amostra['hora_inicio'] = amostra['hora_inicio'].astype(str)
amostra['datetime_partida'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_inicio'])
amostra['hora_fim'] = amostra['hora_fim'].astype(str)
amostra['datetime_chegada'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_fim'])
amostra['id_veiculo'] = amostra['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
amostra_tratada =  amostra[amostra['servico'] == servico]

amostra_tratada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00
...,...,...,...,...,...,...,...,...
403,2023-07-19,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00
404,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00
405,2023-07-19,844,47689,18:18:00,18:33:00,V,2023-07-19 18:18:00,2023-07-19 18:33:00
406,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00


In [4]:
amostra_tratada_2022 = amostra_tratada[amostra_tratada['data'] <= '2022-12-31']
amostra_tratada_2023 = amostra_tratada[amostra_tratada['data'] > '2022-12-31']
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()

print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("A quantidade de viagens na amostra em 2022 é:", amostra_tratada_2022.shape[0])
print("A quantidade de viagens na amostra em 2023 é:", amostra_tratada_2023.shape[0])
print("A amostra é referente aos dias:", datas)
print("A amostra tem dados dos seguintes veículos:", veiculos)

A quantidade total de viagens na amostra é: 280
A quantidade de viagens na amostra em 2022 é: 136
A quantidade de viagens na amostra em 2023 é: 144
A amostra é referente aos dias: ['2022-09-15' '2022-09-20' '2022-09-28' '2023-07-06' '2023-07-11'
 '2023-07-19']
A amostra tem dados dos seguintes veículos: ['47689']


#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [5]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada)
amostra_deduplicada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN
...,...,...,...,...,...,...,...,...,...
403,2023-07-19,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00,NaN
404,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,NaN
405,2023-07-19,844,47689,18:18:00,18:33:00,V,2023-07-19 18:18:00,2023-07-19 18:33:00,NaN
406,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,NaN


In [6]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

0

### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [7]:
# separar as datas da amostra para fazer a query
datas = amostra_deduplicada['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_completa_prod = bd.read_sql(q, from_file=True)
viagem_completa_prod.info() 

viagem_completa_prod['id_veiculo'] = viagem_completa_prod['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_completa_prod      

Downloading: 100%|██████████| 95/95 [00:00<00:00, 226.19rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         95 non-null     object        
 1   servico_informado  95 non-null     object        
 2   sentido            95 non-null     object        
 3   datetime_partida   95 non-null     datetime64[ns]
 4   datetime_chegada   95 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 3.8+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,47689,844,C,2022-09-15 11:14:54,2022-09-15 11:26:24
1,47689,844,C,2022-09-15 13:20:24,2022-09-15 13:32:24
2,47689,844,C,2022-09-15 13:41:54,2022-09-15 13:54:24
3,47689,844,C,2022-09-15 15:09:24,2022-09-15 15:22:54
4,47689,844,C,2022-09-15 17:42:54,2022-09-15 17:56:24
...,...,...,...,...,...
90,47689,844,C,2023-07-06 12:35:06,2023-07-06 12:45:36
91,47689,844,C,2023-07-06 14:32:36,2023-07-06 14:45:06
92,47689,844,C,2023-07-06 13:11:06,2023-07-06 13:21:06
93,47689,844,C,2023-07-06 12:18:06,2023-07-06 12:28:06


In [8]:
datas

array(['2022-09-15', '2022-09-20', '2022-09-28', '2023-07-06',
       '2023-07-11', '2023-07-19'], dtype=object)

Esta etapa cruza dados da amostra com as viagens completas para o serviço 844 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 4 minutos do datetime_partida que consta no amostra.

In [9]:
viagem_completa_prod['datetime_partida'] = pd.to_datetime(viagem_completa_prod['datetime_partida'])
amostra_classificada = check_complete_trips(amostra_deduplicada, viagem_completa_prod, 4)
amostra_classificada


Casos duplicados encontrados no cruzamento de dados:
         data servico id_veiculo_amostra hora_inicio  hora_fim  \
0  2022-09-28     844              47689    10:46:00  10:52:00   
1  2022-09-28     844              47689    10:52:00  11:00:00   

  sentido_amostra datetime_partida_amostra datetime_chegada_amostra  \
0               I      2022-09-28 10:46:00      2022-09-28 10:52:00   
1               V      2022-09-28 10:52:00      2022-09-28 11:00:00   

                                              status servico_informado  \
0  O veículo existe e operou na linha indicada pe...               844   
1  O veículo existe e operou na linha indicada pe...               844   

  id_veiculo_apurada sentido_apurada datetime_partida_apurada  \
0              47689               C      2022-09-28 10:48:24   
1              47689               C      2022-09-28 10:48:24   

  datetime_chegada_apurada  
0      2022-09-28 10:57:24  
1      2022-09-28 10:57:24  


,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-15,844,47689,13:17:00,13:27:00,I,2022-09-15 13:17:00,2022-09-15 13:27:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 13:20:24,2022-09-15 13:32:24
1,2022-09-28,844,47689,10:46:00,10:52:00,I,2022-09-28 10:46:00,2022-09-28 10:52:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 10:48:24,2022-09-28 10:57:24
2,2022-09-28,844,47689,10:52:00,11:00:00,V,2022-09-28 10:52:00,2022-09-28 11:00:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 10:48:24,2022-09-28 10:57:24
3,2022-09-28,844,47689,13:56:00,14:08:00,I,2022-09-28 13:56:00,2022-09-28 14:08:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 13:59:54,2022-09-28 14:12:54
4,2023-07-06,844,47689,10:31:00,10:37:00,V,2023-07-06 10:31:00,2023-07-06 10:37:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2023-07-06 10:32:06,2023-07-06 10:43:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,NaN,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,NaN,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,NaN,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,NaN,NaN,NaN,NaN,NaT,NaT


In [10]:
count_non_nan = amostra_classificada['status'].notna().sum()
count_non_nan

73

In [11]:
# Corrigir linha duplicada
condition = (
    (amostra_classificada['id_veiculo_amostra'] == '47689') & 
    (amostra_classificada['datetime_partida_amostra'] == "2022-09-28 10:52:00")
)

# Update the columns from 'status' till the end for those rows
amostra_classificada.loc[condition, 'status':] = np.nan

amostra_classificada

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-15,844,47689,13:17:00,13:27:00,I,2022-09-15 13:17:00,2022-09-15 13:27:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 13:20:24,2022-09-15 13:32:24
1,2022-09-28,844,47689,10:46:00,10:52:00,I,2022-09-28 10:46:00,2022-09-28 10:52:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 10:48:24,2022-09-28 10:57:24
2,2022-09-28,844,47689,10:52:00,11:00:00,V,2022-09-28 10:52:00,2022-09-28 11:00:00,NaN,NaN,NaN,NaN,NaT,NaT
3,2022-09-28,844,47689,13:56:00,14:08:00,I,2022-09-28 13:56:00,2022-09-28 14:08:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 13:59:54,2022-09-28 14:12:54
4,2023-07-06,844,47689,10:31:00,10:37:00,V,2023-07-06 10:31:00,2023-07-06 10:37:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2023-07-06 10:32:06,2023-07-06 10:43:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,NaN,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,NaN,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,NaN,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,NaN,NaN,NaN,NaN,NaT,NaT


In [12]:
# Convert the 'data' column to datetime format
amostra_classificada['data'] = pd.to_datetime(amostra_classificada['data'])

# Filter for rows where 'data' is in 2022
amostra_2022 = amostra_classificada[amostra_classificada['data'].dt.year == 2022]

# Filter for rows where 'data' is in 2023
amostra_2023 = amostra_classificada[amostra_classificada['data'].dt.year == 2023]


# Count non-NaN values in the 'status' column for each filtered DataFrame
non_nan_count_2022 = amostra_2022['status'].notna().sum()
non_nan_count_2023 = amostra_2023['status'].notna().sum()

print(f"Viagens classificadas em 2022: {non_nan_count_2022}")
print(f"Viagens classificadas em 2022 em %: {round(non_nan_count_2022 / amostra_classificada.shape[0],2)*100} %")

print(f"Viagens classificadas em 2023: {non_nan_count_2023}")
print(f"Viagens classificadas em 2023: {round(non_nan_count_2023 / amostra_classificada.shape[0],2)*100} %")

Viagens classificadas em 2022: 3
Viagens classificadas em 2022 em %: 1.0 %
Viagens classificadas em 2023: 69
Viagens classificadas em 2023: 25.0 %


Foram identificadas 72 das 280 viagens válidas da amostra. Nestes casos, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

#### 3.3 Verificar dados de GPS

In [13]:
amostra_sem_status = amostra_classificada[pd.isna(amostra_classificada['status'])]
amostra_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
2,2022-09-28,844,47689,10:52:00,11:00:00,V,2022-09-28 10:52:00,2022-09-28 11:00:00,NaN,NaN,NaN,NaN,NaT,NaT
73,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN,NaN,NaN,NaN,NaT,NaT
74,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN,NaN,NaN,NaN,NaT,NaT
75,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN,NaN,NaN,NaN,NaT,NaT
76,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,NaN,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,NaN,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,NaN,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,NaN,NaN,NaN,NaN,NaT,NaT


##### 3.3.1 Tabela `registros_status_viagem`

Verificar se estes veículos das viagens não encontradas possuem sinal de GPS para o intervalo da amostra e na linha 663 na tabela `projeto_subsidio_sppo.registros_status_viagem`.
Obs: caso não encontre, procurar tabela `gps_sppo`.


In [14]:
data_n_encontradas = amostra_sem_status['data'].unique()
data_n_encontradas = pd.Series(data_n_encontradas)
data_n_encontradas = data_n_encontradas.dt.strftime('%Y-%m-%d')
data_n_encontradas = data_n_encontradas.values
data_n_encontradas

array(['2022-09-28', '2022-09-15', '2022-09-20', '2023-07-06',
       '2023-07-11', '2023-07-19'], dtype=object)

In [15]:
ids_n_encontrados = amostra_sem_status['id_veiculo_amostra'].unique()
ids_n_encontrados

array(['47689'], dtype=object)

In [16]:
# Cuidado ao executar esta query. Verificar o número de dias acima.
q = f"""
  SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps
FROM
    `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA IN {tuple(data_n_encontradas)}
  AND id_veiculo = "C47689"
"""  

gps_registros_status_viagem = bd.read_sql(q, from_file=True)
gps_registros_status_viagem.info() 

Downloading: 100%|██████████| 2914/2914 [00:00<00:00, 6186.76rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2914 entries, 0 to 2913
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_veiculo     2914 non-null   object        
 1   servico        2914 non-null   object        
 2   timestamp_gps  2914 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 68.4+ KB


In [17]:
gps_registros_status_viagem['id_veiculo'] = gps_registros_status_viagem['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)

Classificar dados encontrados:

In [18]:
def set_status(row, df_check):
    # Filtra o dataframe de checagem pelo id_veiculo e intervalo de tempo
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]

    # Se o dataframe filtrado não tiver registros, retorna a mensagem correspondente
    if filtered_df.empty:
        return "Sem sinal de GPS em registros_status_viagem"

    # Se tiver e o serviço for o mesmo, retorna a mensagem correspondente
    elif filtered_df.iloc[0]['servico'] == row['servico']:
        return "Sinal de GPS encontrado em registros_status_viagem para o mesmo serviço"

    # Se o serviço for diferente, retorna a mensagem informando o número do serviço
    else:
        return f"Sinal de GPS encontrado para veículo em outro serviço: ({filtered_df.iloc[0]['servico']})"

# Aplica a função ao dataframe
amostra_sem_status['status'] = amostra_sem_status.apply(lambda row: set_status(row, gps_registros_status_viagem), axis=1)

amostra_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
2,2022-09-28,844,47689,10:52:00,11:00:00,V,2022-09-28 10:52:00,2022-09-28 11:00:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
73,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,Sem sinal de GPS em registros_status_viagem,NaN,NaN,NaN,NaT,NaT
74,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,Sem sinal de GPS em registros_status_viagem,NaN,NaN,NaN,NaT,NaT
75,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,Sem sinal de GPS em registros_status_viagem,NaN,NaN,NaN,NaT,NaT
76,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,Sem sinal de GPS em registros_status_viagem,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT


##### 3.3.2 Tabela `gps_sppo`

Verificar se existem dados de gps em `gps_sppo` nos casos em que o GPS não foi encontrado na tabela `registros_status_viagem`.

In [19]:
amostra_sem_gps = amostra_sem_status[amostra_sem_status['status'] == "Sem sinal de GPS em registros_status_viagem"]

data_n_encontradas_gps = amostra_sem_gps['data'].unique()
data_n_encontradas_gps = pd.Series(data_n_encontradas_gps)
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

ids_n_encontrados_gps = amostra_sem_gps['id_veiculo_amostra'].unique()

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA IN {tuple(data_n_encontradas_gps)}
  AND id_veiculo = "C47689"
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()
dados_gps['id_veiculo'] = dados_gps['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)

Downloading: 100%|██████████| 10927/10927 [00:01<00:00, 9501.74rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10927 entries, 0 to 10926
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           10927 non-null  object        
 1   servico              10927 non-null  object        
 2   timestamp_gps        10927 non-null  datetime64[ns]
 3   posicao_veiculo_geo  10927 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 341.6+ KB


In [20]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if (filtered_df.iloc[0]['servico'] == row['servico']) and (row['status'] == "Sem sinal de GPS em registros_status_viagem"):
            return "Encontrado sinal de GPS, mas sem dados em registros_status_viagem "
    return row['status']

amostra_sem_status['status'] = amostra_sem_status.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
2,2022-09-28,844,47689,10:52:00,11:00:00,V,2022-09-28 10:52:00,2022-09-28 11:00:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
73,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,"Encontrado sinal de GPS, mas sem dados em regi...",NaN,NaN,NaN,NaT,NaT
74,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,"Encontrado sinal de GPS, mas sem dados em regi...",NaN,NaN,NaN,NaT,NaT
75,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,"Encontrado sinal de GPS, mas sem dados em regi...",NaN,NaN,NaN,NaT,NaT
76,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,"Encontrado sinal de GPS, mas sem dados em regi...",NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT


In [21]:
# juntar todas as linhas e classificar abaixo:
amostra_classificada_com_status = amostra_classificada[pd.notna(amostra_classificada['status'])]
df_final = pd.concat([amostra_classificada_com_status, amostra_sem_status], ignore_index=True)
df_final

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-15,844,47689,13:17:00,13:27:00,I,2022-09-15 13:17:00,2022-09-15 13:27:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 13:20:24,2022-09-15 13:32:24
1,2022-09-28,844,47689,10:46:00,10:52:00,I,2022-09-28 10:46:00,2022-09-28 10:52:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 10:48:24,2022-09-28 10:57:24
2,2022-09-28,844,47689,13:56:00,14:08:00,I,2022-09-28 13:56:00,2022-09-28 14:08:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-28 13:59:54,2022-09-28 14:12:54
3,2023-07-06,844,47689,10:31:00,10:37:00,V,2023-07-06 10:31:00,2023-07-06 10:37:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2023-07-06 10:32:06,2023-07-06 10:43:06
4,2023-07-06,844,47689,10:52:00,10:57:00,V,2023-07-06 10:52:00,2023-07-06 10:57:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2023-07-06 10:53:06,2023-07-06 11:04:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,17:11:00,17:19:00,I,2023-07-19 17:11:00,2023-07-19 17:19:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
276,2023-07-19,844,47689,17:38:00,17:46:00,I,2023-07-19 17:38:00,2023-07-19 17:46:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
277,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
278,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT


In [22]:
# exportar em csv
df_final.to_csv('./../data/output/analise_amostra_pre_solucao.csv', index=False)

# Exemplos (mapas)

Viagem identificada

Veículo: C47689

partida: 2022-09-15 13:17:39	
chegada: 2022-09-15 13:27:09	

In [25]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-07-06"
  AND servico = '844'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.51rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [26]:
q = f"""
SELECT
  id_veiculo,
  servico_informado AS servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-07-06"
  AND servico_informado = '844'
  AND timestamp_gps BETWEEN '2023-07-06 10:32:06'
  AND '2023-07-06 10:43:06'
  AND id_veiculo = 'C47689'
"""
      
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 16/16 [00:00<00:00, 42.55rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           16 non-null     object        
 1   servico              16 non-null     object        
 2   timestamp_gps        16 non-null     datetime64[ns]
 3   posicao_veiculo_geo  16 non-null     object        
 4   status_viagem        16 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 768.0+ bytes


In [208]:
create_trip_map(gps_identificado, shape_identificado)

Viagem não identificada

C47689	28-09-2022 entre 12:22:00 e 12:34:00


In [27]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-28"
  AND servico = '844'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.45rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [28]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-28"
  AND servico = '844'
  AND timestamp_gps BETWEEN "2022-09-28T12:22:00"
  AND "2022-09-28T12:34:00"
  AND id_veiculo = "C47689"
"""

gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 100%|██████████| 24/24 [00:00<00:00, 75.47rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           24 non-null     object        
 1   servico              24 non-null     object        
 2   timestamp_gps        24 non-null     datetime64[ns]
 3   posicao_veiculo_geo  24 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 896.0+ bytes


In [29]:
create_trip_map(gps_n_identificado, shape_n_identificado)

## (2) Gráficos da análise exploratória / análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [30]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '{servico}'
  AND DATA BETWEEN '2022-06-01' AND '2023-08-31'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 381/381 [00:00<00:00, 1043.84rows/s]


In [31]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [32]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: {}".format(servico), font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

# (3) Análise das viagens da amostra x apuradas (pós-solução)

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pos_solucao.csv`

- Os dados reprocessados para o ano de 2022 estão disponíveis no dataset: `rj-smtr-dev.SMTR202211008775_reprocessamento`

Solução: reprocessamento das viagens de 2022 


> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao_solucao	datetime_chegada_apuracao_solucao	status

In [22]:
q = f"""
SELECT
  data,
  id_veiculo,
  servico_informado,
  sentido,
  datetime_partida,
  datetime_chegada
FROM
  `rj-smtr-dev.SMTR202211008775_reprocessamento.viagem_completa`
WHERE
  DATA IN {tuple(datas)}
"""
viagens_reprocessadas = bd.read_sql(q, from_file=True)
viagens_reprocessadas.info()   

viagens_reprocessadas['id_veiculo'] = viagens_reprocessadas['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagens_reprocessadas['data'] = pd.to_datetime(viagens_reprocessadas['data'])

Downloading: 100%|██████████| 60/60 [00:00<00:00, 165.75rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               60 non-null     dbdate        
 1   id_veiculo         60 non-null     object        
 2   servico_informado  60 non-null     object        
 3   sentido            60 non-null     object        
 4   datetime_partida   60 non-null     datetime64[ns]
 5   datetime_chegada   60 non-null     datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 2.9+ KB


In [23]:
amostra_deduplicada['data'] = pd.to_datetime(amostra_deduplicada['data'])
amostra_2022 = amostra_deduplicada[amostra_deduplicada['data'].dt.year == 2022]
amostra_2023 = amostra_deduplicada[amostra_deduplicada['data'].dt.year == 2023]
amostra_2022['status'] = np.nan
amostra_2022

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN
...,...,...,...,...,...,...,...,...,...
259,2022-09-28,844,47689,18:13:00,18:21:00,V,2022-09-28 18:13:00,2022-09-28 18:21:00,NaN
260,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,NaN
261,2022-09-28,844,47689,18:36:00,18:45:00,V,2022-09-28 18:36:00,2022-09-28 18:45:00,NaN
262,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,NaN


In [25]:
amostra_2022_completa_reprocessada = check_complete_trips(amostra_2022, viagens_reprocessadas, 4)
amostra_2022_completa_reprocessada

Não existem casos duplicados no cruzamento de dados.


,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 10:43:54,2022-09-15 10:57:54
1,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 11:05:24,2022-09-15 11:16:54
2,2022-09-15,844,47689,11:22:00,11:29:00,V,2022-09-15 11:22:00,2022-09-15 11:29:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 11:23:54,2022-09-15 11:37:24
3,2022-09-15,844,47689,12:02:00,12:10:00,V,2022-09-15 12:02:00,2022-09-15 12:10:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 12:04:24,2022-09-15 12:16:24
4,2022-09-15,844,47689,12:24:00,12:32:00,V,2022-09-15 12:24:00,2022-09-15 12:32:00,O veículo existe e operou na linha indicada pe...,844,47689,C,2022-09-15 12:26:24,2022-09-15 12:40:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-09-28,844,47689,17:37:00,17:51:00,I,2022-09-28 17:37:00,2022-09-28 17:51:00,NaN,NaN,NaN,NaN,NaT,NaT
132,2022-09-28,844,47689,17:59:00,18:13:00,I,2022-09-28 17:59:00,2022-09-28 18:13:00,NaN,NaN,NaN,NaN,NaT,NaT
133,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,NaN,NaN,NaN,NaN,NaT,NaT
134,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,NaN,NaN,NaN,NaN,NaT,NaT


In [ ]:
# separar as linhas cima com NAN e aplicar a função para 5 minutos!!! Tem viagens que não está pegando!!

In [70]:
amostra_reprocess_2022 = amostra_2022_completa_reprocessada[amostra_2022_completa_reprocessada['data'].dt.year == 2022]
non_nan_count_2022 = amostra_reprocess_2022['status'].notna().sum()

print(f"Viagens classificadas em 2022: {non_nan_count_2022}")
print(f"Viagens classificadas em 2022 em %: {round(non_nan_count_2022 / amostra_2022.shape[0],2)*100} %")

Viagens classificadas em 2022: 54
Viagens classificadas em 2022 em %: 40.0 %


Antes do reprocessamento, apenas 3 viagens foram identificadas em 2022. Após o reprocessamento, foram 54 viagens identificadas das 136 viagens.

Além disso, nota-se que a linha é circular e que diversas viagens que constam na amostra na verdade não são viagens de ida e volta, mas sim viagens circulares. Estas viagens são identificadas na tabela `registros_status_viagem`.


In [71]:
data_n_encontradas_gps = amostra_reprocess_2022['data'].unique()

data_n_encontradas_gps = amostra_sem_gps['data'].unique()
data_n_encontradas_gps = pd.Series(data_n_encontradas_gps)
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

ids_n_encontrados_gps = amostra_reprocess_2022['id_veiculo_amostra'].unique()

q = f"""
  SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps
FROM
    `rj-smtr-dev.SMTR202211008775_reprocessamento.registros_status_viagem`
WHERE
  DATA IN {tuple(data_n_encontradas)}
  AND id_veiculo = "C47689"
"""  

gps_reprocessado = bd.read_sql(q, from_file=True)
gps_reprocessado.info() 

Downloading: 100%|██████████| 1049/1049 [00:00<00:00, 2689.74rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_veiculo     1049 non-null   object        
 1   servico        1049 non-null   object        
 2   timestamp_gps  1049 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 24.7+ KB


In [72]:
gps_reprocessado['id_veiculo'] = gps_reprocessado['id_veiculo'].astype(str)
gps_reprocessado['id_veiculo'] = gps_reprocessado['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
gps_reprocessado['timestamp_gps'] = pd.to_datetime(gps_reprocessado['timestamp_gps'])
gps_reprocessado

,id_veiculo,servico,timestamp_gps
0,47689,844,2022-09-20 18:16:24
1,47689,844,2022-09-20 18:15:24
2,47689,844,2022-09-20 18:12:54
3,47689,844,2022-09-20 18:14:54
4,47689,844,2022-09-20 18:14:24
...,...,...,...
1044,47689,844,2022-09-15 15:56:54
1045,47689,844,2022-09-15 15:54:24
1046,47689,844,2022-09-15 15:56:24
1047,47689,844,2022-09-15 15:54:54


In [73]:
amostra_2022_na = amostra_reprocess_2022[amostra_reprocess_2022['status'].isna()]

amostra_2022_na['id_veiculo_amostra'] = amostra_2022_na['id_veiculo_amostra'].astype(str)
amostra_2022_na['datetime_partida_amostra'] = pd.to_datetime(amostra_2022_na['datetime_partida_amostra'])
amostra_2022_na['datetime_chegada_amostra'] = pd.to_datetime(amostra_2022_na['datetime_chegada_amostra'])
amostra_2022_na

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
54,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN,NaN,NaN,NaN,NaT,NaT
55,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN,NaN,NaN,NaN,NaT,NaT
56,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN,NaN,NaN,NaN,NaT,NaT
57,2022-09-15,844,47689,11:29:00,11:40:00,I,2022-09-15 11:29:00,2022-09-15 11:40:00,NaN,NaN,NaN,NaN,NaT,NaT
58,2022-09-15,844,47689,11:40:00,11:50:00,V,2022-09-15 11:40:00,2022-09-15 11:50:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-09-28,844,47689,17:37:00,17:51:00,I,2022-09-28 17:37:00,2022-09-28 17:51:00,NaN,NaN,NaN,NaN,NaT,NaT
132,2022-09-28,844,47689,17:59:00,18:13:00,I,2022-09-28 17:59:00,2022-09-28 18:13:00,NaN,NaN,NaN,NaN,NaT,NaT
133,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,NaN,NaN,NaN,NaN,NaT,NaT
134,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,NaN,NaN,NaN,NaN,NaT,NaT


In [74]:
# filtered_gps = gps_reprocessado[
#     (gps_reprocessado['id_veiculo'] == "47689") & 
#     (gps_reprocessado['timestamp_gps'] >= "2022-09-15T10:32:00") & 
#     (gps_reprocessado['timestamp_gps'] <= "2022-09-15T10:40:00")
# ]

# filtered_gps

In [75]:
def set_status(row, df_check):
    # Filtra o dataframe de checagem pelo id_veiculo e intervalo de tempo
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]

    # Se o dataframe filtrado não tiver registros, retorna a mensagem correspondente
    if filtered_df.empty:
        return "Sem sinal de GPS em registros_status_viagem"

    # Se tiver e o serviço for o mesmo, retorna a mensagem correspondente
    elif filtered_df.iloc[0]['servico'] == row['servico']:
        return "Sinal de GPS encontrado em registros_status_viagem para o mesmo serviço"

    # Se o serviço for diferente, retorna a mensagem informando o número do serviço
    else:
        return f"Sinal de GPS encontrado para veículo em outro serviço: ({filtered_df.iloc[0]['servico']})"


amostra_2022_na['status'] = amostra_2022_na.apply(lambda row: set_status(row, gps_reprocessado), axis=1)
amostra_2022_na

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
54,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,Sem sinal de GPS em registros_status_viagem,NaN,NaN,NaN,NaT,NaT
55,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
56,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
57,2022-09-15,844,47689,11:29:00,11:40:00,I,2022-09-15 11:29:00,2022-09-15 11:40:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
58,2022-09-15,844,47689,11:40:00,11:50:00,V,2022-09-15 11:40:00,2022-09-15 11:50:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-09-28,844,47689,17:37:00,17:51:00,I,2022-09-28 17:37:00,2022-09-28 17:51:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
132,2022-09-28,844,47689,17:59:00,18:13:00,I,2022-09-28 17:59:00,2022-09-28 18:13:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
133,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
134,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT


In [76]:
amostra_sem_gps = amostra_2022_na[amostra_2022_na['status'] == "Sem sinal de GPS em registros_status_viagem"]

data_n_encontradas_gps = amostra_sem_gps['data'].unique()
data_n_encontradas_gps = pd.Series(data_n_encontradas_gps)
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

ids_n_encontrados_gps = amostra_sem_gps['id_veiculo_amostra'].unique()

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA IN {tuple(data_n_encontradas_gps)}
  AND id_veiculo = "C47689"
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()
dados_gps['id_veiculo'] = dados_gps['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)

Downloading: 100%|██████████| 8640/8640 [00:00<00:00, 8888.92rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           8640 non-null   object        
 1   servico              8640 non-null   object        
 2   timestamp_gps        8640 non-null   datetime64[ns]
 3   posicao_veiculo_geo  8640 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 270.1+ KB


In [77]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if (filtered_df.iloc[0]['servico'] == row['servico']) and (row['status'] == "Sem sinal de GPS em registros_status_viagem"):
            return "Encontrado sinal de GPS, mas sem dados em registros_status_viagem "
    return row['status']

amostra_2022_na['status'] = amostra_2022_na.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_2022_na

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
54,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,"Encontrado sinal de GPS, mas sem dados em regi...",NaN,NaN,NaN,NaT,NaT
55,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
56,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
57,2022-09-15,844,47689,11:29:00,11:40:00,I,2022-09-15 11:29:00,2022-09-15 11:40:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
58,2022-09-15,844,47689,11:40:00,11:50:00,V,2022-09-15 11:40:00,2022-09-15 11:50:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-09-28,844,47689,17:37:00,17:51:00,I,2022-09-28 17:37:00,2022-09-28 17:51:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
132,2022-09-28,844,47689,17:59:00,18:13:00,I,2022-09-28 17:59:00,2022-09-28 18:13:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
133,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT
134,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,Sinal de GPS encontrado em registros_status_vi...,NaN,NaN,NaN,NaT,NaT


In [87]:
df_final_2023 = df_final[df_final['data'].dt.year == 2023]

amostra_reprocess_2022.update(amostra_2022_na)


# passar daqui para baixo mais para baixo
# unir os dois dataframes
tabela_final_status = pd.concat([df_final_2023, amostra_reprocess_2022], ignore_index=True)
tabela_final_status

# classificar as viagens de ida que são inválidas (pq a linha é circular)
tabela_final_status.loc[(tabela_final_status['sentido_amostra'] == 'I') & (tabela_final_status['status'] == 'Sinal de GPS encontrado em registros_status_viagem para o mesmo serviço'), 'status'] = 'Viagem inválida - a linha é circular'

# # as viagens que estavam em registros_status_viagem, não apresentaram a conformidade para ser uma viagem completa.
# tabela_final_status.loc[(tabela_final_status['status'] == 'Sinal de GPS encontrado em registros_status_viagem para o mesmo serviço'), 'status'] = 'Viagem inválida - Viagem não atende a conformidade'

tabela_final_status.to_csv('../data/output/analise_amostra_pos_solucao.csv')

### Exemplos (mapa)

- Mapa de 1 viagem identificada (que não foi identificada antes da
  solução): `analise_mapa_viagem_identificada_pos_solucao.html`
- Mapa de viagens não identificadas (se necessário i.e. teve sinal de
  GPS na linha no período, mas não teve a viagem): `analise_mapa_viagem_nao_identificada_pos_solucao_[descricao].html`

In [83]:
# viagem não encontrada 



# Verificar conformidade quando status for "Sinal de GPS encontrado em registros_status_viagem para o mesmo serviço"




In [84]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2022-09-15'
  AND id_veiculo = "C47689"
  AND timestamp_gps BETWEEN "2022-09-15T11:40:00" AND "2022-09-15T12:00:00"
"""     
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()

dados_gps.to_csv('../scripts/data_graph_test/gps_check.csv')

Downloading: 100%|██████████| 40/40 [00:00<00:00, 110.80rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           40 non-null     object        
 1   servico              40 non-null     object        
 2   timestamp_gps        40 non-null     datetime64[ns]
 3   posicao_veiculo_geo  40 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.4+ KB


In [85]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-15"
  AND servico = '844'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 
shape_n_identificado.to_csv('../scripts/data_graph_test/shape_check.csv')

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.92rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [86]:
create_trip_map(dados_gps, shape_identificado)

NameError: name 'shape_identificado' is not defined